In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,glob,tqdm


In [3]:
df=pd.read_excel("./Data/processing_data_ot_playtime_rating_schedule.xlsx")
print(df.shape)
df.head()

(37368, 20)


,timestemp,month,day,hour,minute,dayofweek,week,schedule_day,schedule_week,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot,quantity
0,2019-01-01 06:00:00,1,1,6,0,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,2099000,0,53
1,2019-01-01 06:00:00,1,1,6,0,1,1,1,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,4371000,0,110
2,2019-01-01 06:20:00,1,1,6,20,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,3262000,1,82
3,2019-01-01 06:20:00,1,1,6,20,1,1,1,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6955000,1,174
4,2019-01-01 06:40:00,1,1,6,40,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6672000,2,167


In [4]:
df["quantity"]=np.round(df["profit"]/df["price"])
pd.options.display.float_format = '{:.1f}'.format
df.head()

,timestemp,month,day,hour,minute,dayofweek,week,schedule_day,schedule_week,playtime,uniquecode,productcode,productname,category,small_category,rating,price,profit,ot,quantity
0,2019-01-01 06:00:00,1,1,6,0,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,2099000,0,53.0
1,2019-01-01 06:00:00,1,1,6,0,1,1,1,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,4371000,0,110.0
2,2019-01-01 06:20:00,1,1,6,20,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,3262000,1,82.0
3,2019-01-01 06:20:00,1,1,6,20,1,1,1,1,20,100346,201079,테이트 여성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6955000,1,174.0
4,2019-01-01 06:40:00,1,1,6,40,1,1,1,1,20,100346,201072,테이트 남성 셀린니트3종,의류,티셔츠/스웨터,0.0,39900,6672000,2,167.0


In [5]:
product_name =set(df["productname"])
len(product_name)

1692

In [6]:
week_rank=df[["schedule_day","productname","ot"]].groupby(["schedule_day","productname"]).agg(["count","max"]).reset_index()
week_rank

schedule_day                productname    ot    
                                             count max
0               1               뉴트리원 다이어트 구미     6   2
1               1          (더블+더블)일월 순면 온열매트     3   2
2               1          (더블+싱글)일월 순면 온열매트     3   2
3               1          (더블사이즈)일월 순면 온열매트     3   2
4               1  (무) 삼익가구 LED 제니비 서랍형 침대 K     6   2
...           ...                        ...   ...  ..
9900           31      헤비추얼 남성 이지라이프 데님팬츠 3종     3   2
9901           31      헤비추얼 여성 이지라이프 데님팬츠 3종     3   2
9902           31           헤스티지 그레이스 양가죽 코트     9   2
9903           31         헤스티지 마론 양가죽 남성집업재킷     6   0
9904           31                  현대어찬 손질갈치     6   2

[9905 rows x 4 columns]

In [7]:
week_rank["week_rank"]=week_rank[("ot","count")]/ (week_rank[("ot","max")]+1)
week_rank

schedule_day                productname    ot     week_rank
                                             count max          
0               1               뉴트리원 다이어트 구미     6   2       2.0
1               1          (더블+더블)일월 순면 온열매트     3   2       1.0
2               1          (더블+싱글)일월 순면 온열매트     3   2       1.0
3               1          (더블사이즈)일월 순면 온열매트     3   2       1.0
4               1  (무) 삼익가구 LED 제니비 서랍형 침대 K     6   2       2.0
...           ...                        ...   ...  ..       ...
9900           31      헤비추얼 남성 이지라이프 데님팬츠 3종     3   2       1.0
9901           31      헤비추얼 여성 이지라이프 데님팬츠 3종     3   2       1.0
9902           31           헤스티지 그레이스 양가죽 코트     9   2       3.0
9903           31         헤스티지 마론 양가죽 남성집업재킷     6   0       6.0
9904           31                  현대어찬 손질갈치     6   2       2.0

[9905 rows x 5 columns]

In [8]:
np.max(week_rank["week_rank"])

7.666666666666667

In [9]:
np.min(week_rank["week_rank"])

0.5

# 3일 묶었을 때


In [10]:
#3일로 묶기
count=0
week=1
week_box = []
old_day = 1
for i in range(df.shape[0]):
    data = df.iloc[i]
    new_dayofweek = data["dayofweek"]
    new_day = data["schedule_day"]
    if(i==0):
        print("처음 count",count)
        if(new_dayofweek!=0):
            count=new_dayofweek
            print("초기 count",count)
    if(old_day !=new_day):
        print(count)
        count+=1
        old_day = new_day
        if(count==3):
            print("새로운 날짜 ",old_day)
            count=0
            week+=1
            print("몇주 ",week)
        week_box.append(week)
        
    else:
        week_box.append(week)
        
       

처음 count 0
초기 count 1
1
2
새로운 날짜  3
몇주  2
0
1
2
새로운 날짜  6
몇주  3
0
1
2
새로운 날짜  9
몇주  4
0
1
2
새로운 날짜  12
몇주  5
0
1
2
새로운 날짜  15
몇주  6
0
1
2
새로운 날짜  18
몇주  7
0
1
2
새로운 날짜  21
몇주  8
0
1
2
새로운 날짜  24
몇주  9
0
1
2
새로운 날짜  27
몇주  10
0
1
2
새로운 날짜  30
몇주  11
0
1
2
새로운 날짜  2
몇주  12
0
1
2
새로운 날짜  5
몇주  13
0
1
2
새로운 날짜  8
몇주  14
0
1
2
새로운 날짜  11
몇주  15
0
1
2
새로운 날짜  14
몇주  16
0
1
2
새로운 날짜  17
몇주  17
0
1
2
새로운 날짜  20
몇주  18
0
1
2
새로운 날짜  23
몇주  19
0
1
2
새로운 날짜  26
몇주  20
0
1
2
새로운 날짜  1
몇주  21
0
1
2
새로운 날짜  4
몇주  22
0
1
2
새로운 날짜  7
몇주  23
0
1
2
새로운 날짜  10
몇주  24
0
1
2
새로운 날짜  13
몇주  25
0
1
2
새로운 날짜  16
몇주  26
0
1
2
새로운 날짜  19
몇주  27
0
1
2
새로운 날짜  22
몇주  28
0
1
2
새로운 날짜  25
몇주  29
0
1
2
새로운 날짜  28
몇주  30
0
1
2
새로운 날짜  31
몇주  31
0
1
2
새로운 날짜  3
몇주  32
0
1
2
새로운 날짜  6
몇주  33
0
1
2
새로운 날짜  9
몇주  34
0
1
2
새로운 날짜  12
몇주  35
0
1
2
새로운 날짜  15
몇주  36
0
1
2
새로운 날짜  18
몇주  37
0
1
2
새로운 날짜  21
몇주  38
0
1
2
새로운 날짜  24
몇주  39
0
1
2
새로운 날짜  27
몇주  40
0
1
2
새로운 날짜  30
몇주  41
0
1
2
새로운 날짜  3
몇주  42
0
1
2
새로운 날짜  6
몇

In [11]:
max(week_box)

122

In [ ]:
df["schedule_threeday"] = week_box

In [19]:
week_rank=df[["schedule_threeday","productname","ot"]].groupby(["schedule_threeday","productname"]).agg(["count","max"]).reset_index()
week_rank["week_rank"]=week_rank[("ot","count")]/ (week_rank[("ot","max")]+1)

print("max:",np.max(week_rank["week_rank"]))
print("min:",np.min(week_rank["week_rank"]))
week_rank

max: 12.0
min: 0.5


schedule_threeday                    productname    ot     week_rank
                                                       count max          
0                     1          (직매입) 종근당건강 아이클리어 루테인     3   2       1.0
1                     1  CERINI by PAT 남성 소프트 기모 릴렉스팬츠     3   2       1.0
2                     1  CERINI by PAT 남성 풀패키지 기모니트 3종     3   2       1.0
3                     1                  NNF 쿠션퍼자켓+베스트     3   2       1.0
4                     1     SPC삼립 한입 호빵 13봉(단일, 팥 13봉)     3   2       1.0
...                 ...                            ...   ...  ..       ...
10590               122            푸마 스트레치 심리스 드로즈 11종     6   2       2.0
10591               122              푸마 시그니처 코튼 트렁크 9종     6   2       2.0
10592               122                   푸마 코튼 언더탑 9종     6   2       2.0
10593               122              피시원 국내산 햇 손질문어 7팩     3   2       1.0
10594               122                한일 대용량 스텐 분쇄믹서기     3   2       1.0

[10595 rows x 5 columns]

In [20]:
week_rank[week_rank["week_rank"]==12]

schedule_threeday         productname    ot     week_rank
                                           count max          
2222                28    헤스티지 그레이스 양가죽 코트    12   0      12.0
2223                28  헤스티지 마론 양가죽 남성집업재킷    12   0      12.0

In [32]:
#3일로 묶었을 때 가장 많이 방송한 횟수 확인 (12번)
df[(df["schedule_threeday"]==28) &( df["productname"]=="헤스티지 그레이스 양가죽 코트" )]

,timestemp,month,day,hour,minute,dayofweek,week,schedule_day,schedule_week,playtime,...,productcode,productname,category,small_category,rating,price,profit,ot,quantity,schedule_threeday
7931,2019-03-22 13:00:00,3,22,13,0,4,12,22,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,18950000,0,95.0,28
7932,2019-03-22 13:20:00,3,22,13,20,4,12,22,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,28255000,0,142.0,28
7934,2019-03-22 13:40:00,3,22,13,40,4,12,22,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,49118000,0,247.0,28
7989,2019-03-22 23:00:00,3,22,23,0,4,12,22,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,23329000,0,117.0,28
7991,2019-03-22 23:20:00,3,22,23,20,4,12,22,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,43140000,0,217.0,28
7994,2019-03-22 23:40:00,3,22,23,40,4,12,22,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.6,199000,48021000,0,241.0,28
8081,2019-03-23 20:20:00,3,23,20,20,5,12,23,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,13665000,0,69.0,28
8083,2019-03-23 20:40:00,3,23,20,40,5,12,23,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,37013000,0,186.0,28
8085,2019-03-23 21:00:00,3,23,21,0,5,12,23,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,42365000,0,213.0,28
8124,2019-03-24 11:00:00,3,24,11,0,6,12,24,12,20,...,200889,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,16484000,0,83.0,28


# 1 주로 묶었을 때

In [33]:
week_rank=df[["schedule_week","productname","ot"]].groupby(["schedule_week","productname"]).agg(["count","max"]).reset_index()
week_rank["week_rank"]=week_rank[("ot","count")]/ (week_rank[("ot","max")]+1)

print("max:",np.max(week_rank["week_rank"]))
print("min:",np.min(week_rank["week_rank"]))
week_rank

max: 18.0
min: 0.5


schedule_week           productname    ot     week_rank
                                         count max          
0                1  (a4)팽현숙의 순대 곱창 전골&볶음     3   2       1.0
1                1   (도냄)무이자 쿠쿠전기밥솥 10인용     9   2       3.0
2                1    (도냄)무이자 쿠쿠전기밥솥 6인용     9   2       3.0
3                1   (도냄)일시불  쿠쿠전기밥솥 6인용     9   2       3.0
4                1   (도냄)일시불 쿠쿠전기밥솥 10인용     9   2       3.0
...            ...                   ...   ...  ..       ...
7582            53   푸마 스트레치 심리스 드로즈 11종     3   2       1.0
7583            53     푸마 시그니처 코튼 트렁크 9종     3   2       1.0
7584            53          푸마 코튼 언더탑 9종     3   2       1.0
7585            53     피시원 국내산 햇 손질문어 7팩     3   2       1.0
7586            53       한일 대용량 스텐 분쇄믹서기     3   2       1.0

[7587 rows x 5 columns]

# 2주로 묶었을 때

In [34]:
#3일로 묶기
count=0
week=1
week_box = []
old_day = 1
for i in range(df.shape[0]):
    data = df.iloc[i]
    new_dayofweek = data["dayofweek"]
    new_day = data["schedule_day"]
    if(i==0):
        print("처음 count",count)
        if(new_dayofweek!=0):
            count=new_dayofweek
            print("초기 count",count)
    if(old_day !=new_day):
        print(count)
        count+=1
        old_day = new_day
        if(count==14):
            print("새로운 날짜 ",old_day)
            count=0
            week+=1
            print("몇주 ",week)
        week_box.append(week)
        
    else:
        week_box.append(week)
        
       

처음 count 0
초기 count 1
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  14
몇주  2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  28
몇주  3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  11
몇주  4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  25
몇주  5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  11
몇주  6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  25
몇주  7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  8
몇주  8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  22
몇주  9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  6
몇주  10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  20
몇주  11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  3
몇주  12
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  17
몇주  13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  1
몇주  14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  15
몇주  15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  29
몇주  16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  12
몇주  17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  26
몇주  18
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  9
몇주  19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  23
몇주  20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
새로운 날짜  7
몇

In [35]:
max(week_box)

27

In [36]:
df["schedule_twoweek"] = week_box

In [37]:
week_rank=df[["schedule_twoweek","productname","ot"]].groupby(["schedule_twoweek","productname"]).agg(["count","max"]).reset_index()
week_rank["week_rank"]=week_rank[("ot","count")]/ (week_rank[("ot","max")]+1)

print("max:",np.max(week_rank["week_rank"]))
print("min:",np.min(week_rank["week_rank"]))
week_rank

max: 30.0
min: 0.5


schedule_twoweek           productname    ot     week_rank
                                            count max          
0                   1  (a4)팽현숙의 순대 곱창 전골&볶음     3   2       1.0
1                   1   (도냄)무이자 쿠쿠전기밥솥 10인용    21   2       7.0
2                   1    (도냄)무이자 쿠쿠전기밥솥 6인용    21   2       7.0
3                   1   (도냄)일시불  쿠쿠전기밥솥 6인용    21   2       7.0
4                   1   (도냄)일시불 쿠쿠전기밥솥 10인용    21   2       7.0
...               ...                   ...   ...  ..       ...
5246               27   푸마 스트레치 심리스 드로즈 11종     3   2       1.0
5247               27     푸마 시그니처 코튼 트렁크 9종     3   2       1.0
5248               27          푸마 코튼 언더탑 9종     3   2       1.0
5249               27     피시원 국내산 햇 손질문어 7팩     3   2       1.0
5250               27       한일 대용량 스텐 분쇄믹서기     3   2       1.0

[5251 rows x 5 columns]

In [38]:
week_rank[week_rank["week_rank"]==30]

schedule_twoweek         productname    ot     week_rank
                                         count max          
972                6    헤스티지 그레이스 양가죽 코트    30   0      30.0
973                6  헤스티지 마론 양가죽 남성집업재킷    30   0      30.0

In [39]:
#3일로 묶었을 때 가장 많이 방송한 횟수 확인 (12번)
df[(df["schedule_twoweek"]==6) &( df["productname"]=="헤스티지 그레이스 양가죽 코트" )]

,timestemp,month,day,hour,minute,dayofweek,week,schedule_day,schedule_week,playtime,...,productname,category,small_category,rating,price,profit,ot,quantity,schedule_threeday,schedule_twoweek
6856,2019-03-12 00:00:00,3,12,0,0,1,11,11,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,13214000,0,66.0,24,6
6858,2019-03-12 00:20:00,3,12,0,20,1,11,11,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,32106000,0,161.0,24,6
6859,2019-03-12 00:40:00,3,12,0,40,1,11,11,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,66977000,0,337.0,24,6
6951,2019-03-12 21:00:00,3,12,21,0,1,11,12,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,25605000,0,129.0,24,6
6953,2019-03-12 21:20:00,3,12,21,20,1,11,12,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,79899000,0,402.0,24,6
6956,2019-03-12 21:40:00,3,12,21,40,1,11,12,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.0,199000,80517000,0,405.0,24,6
7026,2019-03-13 13:00:00,3,13,13,0,2,11,13,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.1,199000,20030000,0,101.0,25,6
7028,2019-03-13 13:20:00,3,13,13,20,2,11,13,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.2,199000,39879000,0,200.0,25,6
7030,2019-03-13 13:40:00,3,13,13,40,2,11,13,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.2,199000,61707000,0,310.0,25,6
7184,2019-03-14 22:00:00,3,14,22,0,3,11,14,11,20,...,헤스티지 그레이스 양가죽 코트,의류,아우터,0.2,199000,24470000,0,123.0,25,6
